In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score, confusion_matrix, zero_one_loss
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

import os
os.chdir('..')  # Go up one directory level
import tensorflow
import tensorflow as tf
from tensorflow import keras

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

from tensorflow.keras import layers
from tensorflow.keras.layers import Layer, Add, InputSpec, Input, BatchNormalization, Dropout,Conv2D, Conv2DTranspose,AveragePooling2D, MaxPooling2D,Dense, Flatten, Reshape, Concatenate, Embedding, Lambda

import tensorflow.keras.backend as K
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Dropout
from TAABLM_mechanism import GlobalContextLayer, AttentionMechanism, custom_objects

from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.initializers import RandomNormal

!pwd

/var/folders/td/w85k3fwn2nn5xffjktw34x_m0000gn/T/ipykernel_76091/4096949599.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


/Users/tuongdoanvanngoc/Documents/Y3T3/378/AMS-Project/callai2/MyProject


In [2]:
from TAABLM_mechanism import GlobalContextLayer, AttentionMechanism, custom_objects
from Preprocessing_Data import load_and_preprocess_data, preprocess_texts, load_glove_embeddings

In [3]:
# datapath = data_dir + 'CallAIDataCorpus2.csv'
datapath = 'data/Engagement_subsection.csv'

data,texts, unique_labels = load_and_preprocess_data(datapath)

In [4]:
# Preprocess texts
padded_sequences, tokenizer, X_tfidf, word_index, index_word = preprocess_texts(texts)

# Load GloVe embeddings
embedding_dim = 50
vocab_size = 10000
max_seq_len=100

embedding_matrix = load_glove_embeddings(vocab_size, embedding_dim, word_index)

In [7]:
# Assuming data['Text'], data['Category'], and tf_idf_tensor are available
X_train, X_test, y_train, y_test = train_test_split(
    padded_sequences, data['Acknowledgment and Gratitude'], test_size=0.2, random_state=42
)

# Split tf_idf_tensor separately to align with X_train and X_test
X_tfidf_train, X_tfidf_test = train_test_split(
    X_tfidf, test_size=0.2, random_state=42
)

In [9]:
import tensorflow as tf
from tensorflow.keras.models import model_from_json
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Specify the directory containing the saved model files
dir_name = 'TAABLM_model'

# 1. Load the model architecture from the saved JSON file
with open(os.path.join(dir_name, 'TAABLM_Section_architecture.json'), 'r') as json_file:
    model_json = json_file.read()

# Recreate the model from the JSON string
model = model_from_json(model_json)


# 3. Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), 
              loss="sparse_categorical_crossentropy", 
              metrics=["accuracy"])

# 4. Define EarlyStopping callback to avoid overfitting
early_stopping = EarlyStopping(
    monitor="val_loss", patience=3, restore_best_weights=True, verbose=1
)

# Train the model
history = model.fit(
    [X_train, X_tfidf_train],  # Training inputs
    y_train,                   # Training labels
    validation_data=([X_test, X_tfidf_test], y_test),  # Validation inputs and labels
    epochs=10,                 # Number of epochs
    batch_size=32,             # Batch size
    callbacks=[early_stopping]  # Early stopping callback
)

Epoch 1/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 75ms/step - accuracy: 0.3663 - loss: 3.3817 - val_accuracy: 0.7281 - val_loss: 2.8799
Epoch 2/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.7670 - loss: 2.3422 - val_accuracy: 0.7281 - val_loss: 2.2277
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.7311 - loss: 1.7521 - val_accuracy: 0.7281 - val_loss: 1.7497
Epoch 4/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - accuracy: 0.7716 - loss: 1.3482 - val_accuracy: 0.7281 - val_loss: 1.3939
Epoch 5/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.8341 - loss: 1.0959 - val_accuracy: 0.7982 - val_loss: 1.1540
Epoch 6/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.8560 - loss: 0.8343 - val_accuracy: 0.7982 - val_loss: 0.9942
Epoch 7/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.9080 - loss: 0.6460 - val_accuracy: 0.8070 - val_loss: 0.8622
Epoch 8/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.9236 - loss: 0.5329 - val_accuracy: 0.8421 - v

In [11]:
# Change the directory
new_directory = "TAABLM_model"
os.chdir(new_directory)

# Specify the directory to create
dir_name = 'Engagement'  # Replace with your desired path

# Create the directory
os.makedirs(dir_name, exist_ok=True)
model.save(os.path.join(dir_name, 'E_Acknowledge.h5'))